In [3]:
import pandas as pd

df = pd.read_csv("Datathon Dataset.csv")

def correct_misaligned_data(df):
    for index, row in df.iterrows():
        try:
            pd.to_datetime(row["Date"])
        except Exception:
            df.at[index, "Infrastructure_Machineries"] = row["Date"]
            df.at[index, "Date"] = ""

    return df

df_corrected = correct_misaligned_data(df)

df_corrected.to_csv("Datathon_Dataset_Corrected.csv", index=False)


In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

file_path = "Datathon_Dataset_Corrected.csv"  
df = pd.read_csv(file_path)

df["Infrastructure_Machineries"].replace("invalid_data", np.nan, inplace=True)

infra_machinery_mapping = {val: idx for idx, val in enumerate(df["Infrastructure_Machineries"].dropna().unique())}
df["Infra_Machinery_Encoded"] = df["Infrastructure_Machineries"].map(infra_machinery_mapping)

imputer = KNNImputer(n_neighbors=3, weights="uniform")
df["Infra_Machinery_Encoded"] = imputer.fit_transform(df[["Infra_Machinery_Encoded"]])

reverse_mapping = {v: k for k, v in infra_machinery_mapping.items()}
df["Infrastructure_Machineries"] = df["Infra_Machinery_Encoded"].round().map(reverse_mapping)

df.drop(columns=["Infra_Machinery_Encoded"], inplace=True)

output_file_path = "Datathon_Dataset_Pre.csv"
df.to_csv(output_file_path, index=False)

print(f"Preprocessed dataset saved as: {output_file_path}")


Preprocessed dataset saved as: Datathon_Dataset_Pre.csv


C:\Users\mail2\AppData\Local\Temp\ipykernel_76660\1176732941.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Infrastructure_Machineries"].replace("invalid_data", np.nan, inplace=True)


In [9]:
df['Market_Share'] = df['Market_Share'].replace(35000, 35)

In [11]:
df

,Unnamed: 0,Un_Named,Date,Infrastructure_Machineries,Daily_Sales _Percentage,Daily_Sales_Quantity,Market_Share,Political,Marketing,Budget,Customer_Id,Region
0,0,0,NaN,Skid Steer Loaders,-0.034464,3534,35,1,1,5000.56,CUST00001,Sherrichester
1,1,1,1/1/2019,Excavators(crawler),0.034464,80,35,1,1,5000.56,CUST00002,North Ryanstad
2,2,2,1/1/2019,Loaders (Wheeled),0.034464,70,35,1,1,5000.56,CUST00003,South Christophermouth
3,3,3,1/1/2019,Skid Steer Loaders,0.034464,70,35,1,1,5000.56,CUST00004,Juliashire
4,4,4,1/1/2019,Compactors,0.034464,68,35,1,1,5000.56,CUST00005,Davidberg
...,...,...,...,...,...,...,...,...,...,...,...,...
2190,2190,0,NaN,Skid Steer Loaders,0.034464,79,35,1,1,5000.56,CUST02191,East Dominiqueshire
2191,2191,1,1/1/2019,Excavators(crawler),0.034464,67,35,1,1,5000.56,CUST02192,Troyside
2192,2192,2,1/1/2019,Loaders (Wheeled),0.034464,70,35,1,1,5000.56,CUST02193,Dodsonport
2193,2193,3,1/1/2019,Skid Steer Loaders,0.034464,89,35,1,1,5000.56,CUST02194,North Johnland


In [13]:
negative_count = (df['Daily_Sales_Quantity'] < 0).sum()
print(f"Number of rows with negative Daily_Sales_Quantity: {negative_count}")

Number of rows with negative Daily_Sales_Quantity: 73


In [17]:
#CORRELATION
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

correlation = df[['Daily_Sales_Quantity', 'Daily_Sales _Percentage']].corr()
print("Correlation Matrix:\n", correlation)

train_data = df[df['Daily_Sales_Quantity'] <= 3000]

X = train_data[['Daily_Sales_Percentage']]  
y = train_data['Daily_Sales_Quantity']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")

outlier_mask = df['Daily_Sales_Quantity'] > 3000 
df.loc[outlier_mask, 'Daily_Sales_Quantity'] = model.predict(df.loc[outlier_mask, ['Daily_Sales_Percentage']])

df['Daily_Sales_Quantity'] = df['Daily_Sales_Quantity'].clip(lower=1)

print("Updated dataset with corrected values")


Correlation Matrix:
                          Daily_Sales_Quantity  Daily_Sales _Percentage
Daily_Sales_Quantity                 1.000000                -0.067742
Daily_Sales _Percentage             -0.067742                 1.000000


KeyError: "None of [Index(['Daily_Sales_Percentage'], dtype='object')] are in the [columns]"

In [21]:
import pandas as pd

df = pd.read_csv("Processed_Datathon_Dataset.csv") 

df['Daily_Sales_Quantity'] = df['Daily_Sales_Quantity'].astype(int)

df['Market_Share'] = 35

df.to_csv("Correct_Processed.csv", index=False) 

print("Dataset processing complete. File saved as 'processed_file.csv'")


Dataset processing complete. File saved as 'processed_file.csv'


In [28]:
pip install catboost


  Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.8/101.7 MB 1.5 MB/s eta 0:01:10
   ---------------------------------------- 0.8/101.7 MB 1.5 MB/s eta 0:01:10
   ---------------------------------------- 1.0/101.7 MB 1.1 MB/s eta 0:01:28
    --------------------------------------- 1.3/101.7 MB 1.2 MB/s eta 0:01:27
    --------------------------------------- 1.3/101.7 MB 1.2 MB/s eta 0:01:27
    --------------------------------------- 1.6/101.7 MB 976.0 kB/s eta 0:01:43
    --------------------------------------- 1.6/101.7 MB 976.0 kB/s eta 0:01:43
    --------------------------------------- 1.8/101.7 MB 959.1 kB/s eta 0:01:45
    -------------

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import numpy as np

file_path = "dataset_Tele Handlers .csv" 
df = pd.read_csv(file_path)

drop_columns = ["Unnamed: 0", "Un_Named", "Date", "Infrastructure_Machineries", "Customer_Id", "Region"]
df_cleaned = df.drop(columns=drop_columns)

X = df_cleaned.drop(columns=["Daily_Sales_Quantity"])
y = df_cleaned["Daily_Sales_Quantity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


catboost_model = CatBoostRegressor(n_estimators=200, learning_rate=0.05, verbose=0, random_state=42)


catboost_model.fit(X_train, y_train)
y_pred = catboost_model.predict(X_test)


results = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R2 Score": r2_score(y_test, y_pred)
}


print("\nCatBoost Performance:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")



CatBoost Performance:
MAE: 8.1071
RMSE: 9.9452
R2 Score: 0.8642


In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import numpy as np

file_path = "dataset_Loaders (Wheeled).csv" 
df = pd.read_csv(file_path)

drop_columns = ["Unnamed: 0", "Un_Named", "Date", "Infrastructure_Machineries", "Customer_Id", "Region"]
df_cleaned = df.drop(columns=drop_columns)

A = df_cleaned.drop(columns=["Daily_Sales_Quantity"])
b = df_cleaned["Daily_Sales_Quantity"]

A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=42)

Wheeled = CatBoostRegressor(n_estimators=200, learning_rate=0.05, verbose=0, random_state=42)

Wheeled.fit(A_train, b_train)
b_pred = Wheeled.predict(A_test)

results = {
    "MAE": mean_absolute_error(b_test, b_pred),
    "RMSE": np.sqrt(mean_squared_error(b_test, b_pred)),
    "R2 Score": r2_score(b_test, b_pred)
}

print("\nCatBoost Performance:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")



CatBoost Performance:
MAE: 8.0915
RMSE: 10.3002
R2 Score: 0.8529


In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import numpy as np

file_path = "dataset_Excavators(crawler).csv" 
df = pd.read_csv(file_path)

drop_columns = ["Unnamed: 0", "Un_Named", "Date", "Customer_Id", "Region", "Infrastructure_Machineries"]
df_cleaned = df.drop(columns=drop_columns)

C = df_cleaned.drop(columns=["Daily_Sales_Quantity"])
d = df_cleaned["Daily_Sales_Quantity"]

C_train, C_test, d_train, d_test = train_test_split(C, d, test_size=0.2, random_state=42)

Excavator = CatBoostRegressor(n_estimators=200, learning_rate=0.05, verbose=0, random_state=42)

Excavator.fit(C_train, d_train)
d_pred = Excavator.predict(C_test)

results = {
    "MAE": mean_absolute_error(d_test, d_pred),
    "RMSE": np.sqrt(mean_squared_error(d_test, d_pred)),
    "R2 Score": r2_score(d_test, d_pred)
}

print("\nCatBoost Performance:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")


CatBoost Performance:
MAE: 8.6772
RMSE: 10.7546
R2 Score: 0.8530


In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import numpy as np

file_path = "dataset_Skid Steer Loaders.csv" 
df = pd.read_csv(file_path)

drop_columns = ["Unnamed: 0", "Un_Named", "Date", "Customer_Id", "Region", "Infrastructure_Machineries"]
df_cleaned = df.drop(columns=drop_columns)

E = df_cleaned.drop(columns=["Daily_Sales_Quantity"])
f = df_cleaned["Daily_Sales_Quantity"]

E_train, E_test, f_train, f_test = train_test_split(E, f, test_size=0.2, random_state=42)

SkidSteer = CatBoostRegressor(n_estimators=200, learning_rate=0.05, verbose=0, random_state=42)

SkidSteer.fit(E_train, f_train)
f_pred = SkidSteer.predict(E_test)

results = {
    "MAE": mean_absolute_error(f_test, f_pred),
    "RMSE": np.sqrt(mean_squared_error(f_test, f_pred)),
    "R2 Score": r2_score(f_test, f_pred)
}

print("\nCatBoost Performance:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")


CatBoost Performance:
MAE: 8.6708
RMSE: 10.8444
R2 Score: 0.8262


In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import numpy as np

file_path = "dataset_Compactors.csv" 
df = pd.read_csv(file_path)

drop_columns = ["Unnamed: 0", "Un_Named", "Date", "Customer_Id", "Region", "Infrastructure_Machineries"]
df_cleaned = df.drop(columns=drop_columns)

G = df_cleaned.drop(columns=["Daily_Sales_Quantity"])
h = df_cleaned["Daily_Sales_Quantity"]

G_train, G_test, h_train, h_test = train_test_split(G, h, test_size=0.2, random_state=42)

Compactors = CatBoostRegressor(n_estimators=200, learning_rate=0.05, verbose=0, random_state=42)

Compactors.fit(G_train, h_train)
h_pred = Compactors.predict(G_test)

results = {
    "MAE": mean_absolute_error(h_test, h_pred),
    "RMSE": np.sqrt(mean_squared_error(h_test, h_pred)),
    "R2 Score": r2_score(h_test, h_pred)
}

print("\nCatBoost Performance:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")


CatBoost Performance:
MAE: 8.2718
RMSE: 10.4788
R2 Score: 0.8677


In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import numpy as np

file_path = "dataset_Backhoe Loader.csv" 
df = pd.read_csv(file_path)

drop_columns = ["Unnamed: 0", "Un_Named", "Date", "Customer_Id", "Region", "Infrastructure_Machineries"]
df_cleaned = df.drop(columns=drop_columns)

I = df_cleaned.drop(columns=["Daily_Sales_Quantity"])
j = df_cleaned["Daily_Sales_Quantity"]

I_train, I_test, j_train, j_test = train_test_split(I, j, test_size=0.2, random_state=42)

Backhoe = CatBoostRegressor(n_estimators=200, learning_rate=0.05, verbose=0, random_state=42)

Backhoe.fit(I_train, j_train)
j_pred = Backhoe.predict(I_test)

results = {
    "MAE": mean_absolute_error(j_test, j_pred),
    "RMSE": np.sqrt(mean_squared_error(j_test, j_pred)),
    "R2 Score": r2_score(j_test, j_pred)
}

print("\nCatBoost Performance:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")



CatBoost Performance:
MAE: 12.6723
RMSE: 15.3277
R2 Score: 0.5908


In [115]:
# Collect predictions
predictions = []
skipped_rows = []

for index, row in df.iterrows():
    infra_type = row["Infrastructure_Machineries"]  # Get the infrastructure type
    model = model_mapping.get(infra_type)  # Get the corresponding model

    if model:
        try:
            input_data = row.drop(["Date", "Customer_Id", "Region", "Infrastructure_Machineries"]).values.reshape(1, -1)
            
            pred = model.predict(input_data)
            predictions.append(pred[0])  # Assuming a single output per row
        
        except Exception as e:
            print(f"Error predicting row {index}: {e}")
            skipped_rows.append(index)

    else:
        print(f"Skipping row {index}: No model found for {infra_type}")
        skipped_rows.append(index)

print("Predictions:", predictions)
print("Skipped Rows:", skipped_rows)


Skipping row 5: No model found for Tele Handlers 
Predictions: [70.33446624377049, 68.46492361106971, 69.05582887141507, 63.24749875723337, 66.72004645776296]
Skipped Rows: [5]


In [143]:
import pandas as pd

# Load the CSV file
file_path = "input_data.csv"
df = pd.read_csv(file_path)

# Dictionary mapping infrastructure types to their corresponding models
model_mapping = {
    "Tele Handlers ": catboost_model,
    "Loaders (Wheeled)": Wheeled,
    "Excavators(crawler)": Excavator,
    "Skid Steer Loaders": SkidSteer,
    "Compactors": Compactors,
    "Backhoe Loader": Backhoe,





}

# Collect predictions
predictions = []
for _, row in df.iterrows():
    infra_type = row["Infrastructure_Machineries"]  # Get the infrastructure type
    model = model_mapping.get(infra_type)  # Get the corresponding model
    if model:
        input_data = row.drop(["Date", "Customer_Id", "Region", "Infrastructure_Machineries"]).values.reshape(1, -1)

          #drop_columns = ["Unnamed: 0", "Un_Named", ]

        pred = model.predict(input_data)
        predictions.append(pred[0])  # Assuming a single output per row

print(predictions)  # List of predictions

[70.33446624377049, 68.46492361106971, 69.05582887141507, 63.24749875723337, 66.72004645776296, 63.24749875723337]


In [145]:
max_demand=predictions

In [147]:
max_demand

[70.33446624377049,
 68.46492361106971,
 69.05582887141507,
 63.24749875723337,
 66.72004645776296,
 63.24749875723337]

In [149]:
max_demand = [round(x) for x in max_demand]

print(max_demand)

[70, 68, 69, 63, 67, 63]


In [151]:
import numpy as np
from scipy.optimize import linprog

# Machine names

machines = ["Compactors", "Loaders (Wheeled)", "Skid Steer Loaders", "Backhoe Loader", "Excavators (crawler)", "Tele Handlers"]

# Space per machine 
spaces = [12, 20, 10, 15, 25, 18]


# Forecasted demand (max units that might be sold)
 

# Total warehouse space limit (cubic meters)
warehouse_capacity = 5000  

#  BEFORE OPTIMIZATION: Check if total space exceeds 5000 cu. meters
total_space_needed = sum(np.array(spaces) * np.array(max_demand))
print("\n BEFORE OPTIMIZATION: Forecasted Demand")
for i in range(len(machines)):
    print(f"{machines[i]}: {max_demand[i]} units")

print(f"Total Space Needed: {total_space_needed} cubic meters")

if total_space_needed > warehouse_capacity:
    print(" Warning: Storage Exceeds 5000 cubic meters! Optimization required.\n")
else:
    print(" Storage is within limits. No need for optimization.\n")


#  OPTIMIZATION USING LINEAR PROGRAMMING
c = -np.array(max_demand)  # Convert to negative since linprog minimizes
A = [spaces]  # Constraint: total space used must be ≤ 5000
b = [warehouse_capacity]  # Space limit
x_bounds = [(0, d) for d in max_demand]  # Each machine stock is between 0 and demand

# Solve optimization
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Extract optimized stock levels
optimized_inventory = np.round(result.x).astype(int)  # Convert to integer

#  AFTER OPTIMIZATION: Print the adjusted inventory
print("\n AFTER OPTIMIZATION: Allocated Inventory")
total_optimized_space = sum(np.array(optimized_inventory) * np.array(spaces))

for i in range(len(machines)):
    print(f"{machines[i]}: {optimized_inventory[i]} units")

print(f" Total Space Used: {total_optimized_space} cubic meters (Max: {warehouse_capacity})")


 BEFORE OPTIMIZATION: Forecasted Demand
Compactors: 70 units
Loaders (Wheeled): 68 units
Skid Steer Loaders: 69 units
Backhoe Loader: 63 units
Excavators (crawler): 67 units
Tele Handlers: 63 units
Total Space Needed: 6644 cubic meters


 AFTER OPTIMIZATION: Allocated Inventory
Compactors: 70 units
Loaders (Wheeled): 68 units
Skid Steer Loaders: 69 units
Backhoe Loader: 63 units
Excavators (crawler): 1 units
Tele Handlers: 63 units
 Total Space Used: 4994 cubic meters (Max: 5000)
